In [ ]:
import db_connect
sql = db_connect.DatabaseConnect('CS581GROUP6','localhost','root','qwerty','3306')

In [ ]:
class Evaluation(object):
    def __init__(self, start_time='2016-06-10 08:00:00', stop_time='2016-06-10 22:00:00'):
        self.start_time = start_time
        self.stop_time=stop_time
    
    def evalute_dist_saved(self):
        df = sql.query("""select tbl_1.poolingWindowID, (sum_indv_rides - sum_shared_rides)/(sum_indv_rides) as percentage_saved from (select pw.poolingWindowID, sum(indvRideDist) as sum_indv_rides from REQUESTS as rq 
                          inner join POOLING_WINDOW as pw on rq.poolingwindowid = pw.poolingwindowid  
                          where pw.poolingstarttime >= str_to_date('{}','%%Y-%%m-%%d %%H:%%i:%%S') 
                          and pw.poolingstarttime <= str_to_date('{}','%%Y-%%m-%%d %%H:%%i:%%S') 
                          group by pw.poolingwindowId) tbl_1 inner join (
                          select poolingWindowID, sum(sum_rides) as sum_shared_rides from (select poolingWindowID, rideID, 
                          max(sharedRideDist) as sum_rides from RIDES group by poolingWindowID, rideID order by 
                          poolingWindowID, rideID) as tbl group by poolingwindowID) as tbl_2 
                          on tbl_1.poolingWindowID = tbl_2.poolingWindowID;""".format(self.start_time, self.stop_time))
        ax = df.plot(kind='line', figsize=(10,5))
        ax.set_xlabel("Pools")
        ax.set_ylabel("% Distance Saved")
        df.fillna(0, inplace=True)
        self.saved_dist = sum(df.percentage_saved)/len(df.percentage_saved)
        print('saved_dist: ',self.saved_dist)
        return df
    
    def evalute_trips_saved(self):
        df = sql.query("""select pw.poolingwindowid, (count(reqid) - count(distinct rideid))/count(reqid) as saved_rides from rides rd inner join pooling_window as pw on 
                          rd.poolingwindowid = pw.poolingwindowid where 
                          pw.poolingstarttime >= str_to_date('{}','%%Y-%%m-%%d %%H:%%i:%%S') 
                          and pw.poolingstarttime <= str_to_date('{}','%%Y-%%m-%%d %%H:%%i:%%S') group by pw.poolingwindowid;""".format(self.start_time, self.stop_time))
        #df=df.astype('int')
        ax = df.plot(kind='line', figsize=(10,5))
        ax.set_xlabel("Pools")
        ax.set_ylabel("% saved_rides")
        df.fillna(0, inplace=True)
        self.saved_trips = sum(df.saved_rides)/len(df.saved_rides)
        print('saved_trips: ',self.saved_trips)
        return df
    
        
    def eval_pool_compute_time(self):
        df = sql.query("""select poolingwindowid, poolingWindowComputeTime from POOLING_WINDOW as pw where pw.poolingstarttime >= str_to_date('{}','%%Y-%%m-%%d %%H:%%i:%%S') and pw.poolingstarttime <= str_to_date('{}','%%Y-%%m-%%d %%H:%%i:%%S') and pw.poolingWindowComputeTime is not null;""".format(self.start_time, self.stop_time))
        df.poolingWindowComputeTime = df.poolingWindowComputeTime.astype('int')
        df.fillna(0, inplace=True)
        ax = df.plot(kind='line', figsize=(10,5))
        ax.set_xlabel("Pools")
        ax.set_ylabel("Computation Time")
        self.avg_compute_time = sum(df.poolingWindowComputeTime)/len(df.poolingWindowComputeTime)
        print('avg_compute_time: ',self.avg_compute_time)
        return df

In [ ]:
a = Evaluation('2016-04-17 08:00:00', '2016-04-17 22:00:00')
df = a.evalute_dist_saved()
df_2 = a.evalute_trips_saved()

In [ ]:
df_3= a.eval_pool_compute_time()

In [ ]:
df

In [ ]:
val = df['poolingWindowID'].where(df['poolingWindowID'] == 130)